In [1]:
import tensorflow as tf
from data_loader import SamplesDataLoader

root_dir = '/Users/jost/Downloads/SPADES'

/opt/anaconda3/envs/NeuroTUM/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# def generate_dataset(data_loader, batch_size, shuffled = True):
#     indexes = range()
#     if shuffled:

data_loader = SamplesDataLoader(root_dir=root_dir, dataset_type="synthetic")
print(len(data_loader.samples))

300


In [3]:
def keras_data_generator(data_loader, batch_size=32, shuffle=True):
    # Shuffle indices if required
    indices = np.arange(len(data_loader.samples))
    if shuffle:
        np.random.shuffle(indices)
    
    # Generator function to yield batches
    for start_idx in range(0, len(indices), batch_size):
        batch_indices = indices[start_idx:start_idx + batch_size]
        
        # Collect events and labels for the batch
        batch_events = []
        batch_labels = []
        for idx in batch_indices:
            events, labels = data_loader[idx]
            batch_events.append(events)
            batch_labels.append(labels)
        
        # Convert to tensors for Keras
        batch_events = tf.ragged.constant(batch_events, dtype=tf.float64)
        batch_labels = tf.ragged.constant(batch_labels, dtype=tf.float64 if batch_labels[0] is not None else tf.float32)
        
        yield batch_events, batch_labels

In [4]:

# Example usage with tf.data.Dataset
data_loader = SamplesDataLoader(root_dir=root_dir, dataset_type='synthetic')
dataset = tf.data.Dataset.from_generator(
    lambda: keras_data_generator(data_loader, batch_size=32),
    output_signature=(
        tf.RaggedTensorSpec(shape=[None, None, 4], dtype=tf.float64),
        tf.RaggedTensorSpec(shape=[None, 8], dtype=tf.float64)
    )
)